In [1]:
import argparse
import time
import cv2
import numpy as np
from PIL import Image
import imutils
import gzip
import cPickle
import pickle
import random

In [2]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)
    
    def predict(self,image):
        value=np.argmax(self.feedforward(image))
        return value

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))


In [3]:
def inverte(imagem):
    imagem = (255-imagem)
    return imagem

In [4]:
output= open('C:/Users/Piyush.Gupta/Desktop/Scorecard image processing/model-95.3.pkl', 'rb')
net= cPickle.load(output)

In [5]:
def sliding_window(image, stepSize, windowSize):
    # slide a window across the image
    for y in xrange(0, image.shape[0], stepSize):
        for x in xrange(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [6]:
def RepInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [7]:
def Reshaping(Img):
    
    old_im=Image.fromarray(Img)
    old_size = old_im.size

    new_size = (28, 28)
    new_im = Image.new("RGB", new_size)   ## luckily, this is already black!
    new_im.paste(old_im, ((new_size[0]-old_size[0])/2,
                          (new_size[1]-old_size[1])/2))

    im=np.array(new_im)
    im=cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    return im

In [12]:
image=cv2.imread("13pic.png",0)
#read the image
height, width= image.shape
test = cv2.medianBlur(image,3)
test = cv2.adaptiveThreshold(test,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2) 
copy=np.reshape(test, (height*width, 1))

#check whther sticker is green or red, if green then proceed
if np.count_nonzero(copy)>0.85*(height*width):
    blur = cv2.GaussianBlur(image,(3,3),0)  # use gaussian blur to reduce noise in the image
    v=int(np.median(image))  #take median of the img
    sigma=0.33
    lower = max(0, (1.0 - sigma) * v)
    lower=int(lower)
    upper = min(255, (1.0 + sigma) * v)
    upper=int(upper)
    edges = cv2.Canny(blur,lower-200,upper)

    cv2.imshow('ghi',edges)
    cv2.waitKey(0)
    
    #find the edges and differenttiate whether it ie 2 digit or 1 digit
    
    (cnts, _) = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse =True)[:10]
    print len(cnts)
    if len(cnts)==1:
        cnt1=cnts[0]
        x,y,w,h = cv2.boundingRect(cnt1)
        crop=image[0:y+h+8,x:x+w]
        #crop the text
        #cv2.imshow('ghi',crop)
        #cv2.waitKey(0)
        img = cv2.medianBlur(crop,3)
        img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2) 
        #smooth the image
        #cv2.imshow('ghi',img)
        #cv2.waitKey(0)
        img=inverte(img)
        #change black to white and vcie versa
        #cv2.imshow('ghi',img)
        #cv2.waitKey(0)
        img=Reshaping(img)
        #change the shape to 28*28
        #cv2.imshow('ghi',img)
        #cv2.waitKey(0)
        img=np.reshape(img, (784, 1))
        #make 1-d array
        result=net.predict(img)
        print result

    elif len(cnts)>1:
        cnts = sorted(cnts, key = cv2.contourArea, reverse =False)[:10]
        cnt1=cnts[0]
        x,y,w,h = cv2.boundingRect(cnt1)
        crop=image[0:y+h+8,x:x+w]
        img = cv2.medianBlur(crop,3)
        img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2) 
        img=inverte(img)
        img=Reshaping(img)
        img=np.reshape(img, (784, 1))
        result=net.predict(img)
        print result

        cnt2=cnts[1]
        x,y,w,h = cv2.boundingRect(cnt2)
        crop=image[0:y+h+8,x:x+w]
        img = cv2.medianBlur(crop,3)
        img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2) 
        img=inverte(img)
        img=Reshaping(img)
        img=np.reshape(img, (784, 1))
        result=net.predict(img)
        print result
else:
    print "more black content on image"
    image1=inverte(image)    
    image1 = cv2.medianBlur(image1,3)
    #img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2) 
    ret,image1 = cv2.threshold(image1,175,255,cv2.THRESH_BINARY) # to find threshold on the basis of black white intensity
    cv2.imshow('def',image1)
    cv2.waitKey(0)
    
    blur = cv2.GaussianBlur(image1,(3,3),0)  # use gaussian blur to reduce noise in the image
    v=int(np.median(image1))  #take median of the img
    sigma=0.33
    lower = max(0, (1.0 - sigma) * v)
    lower=int(lower)
    upper = min(255, (1.0 + sigma) * v)
    upper=int(upper)
    edges = cv2.Canny(blur,lower-200,upper)

    cv2.imshow('ghi',edges)
    cv2.waitKey(0)
    
    #find the edges and differenttiate whether it ie 2 digit or 1 digit
    
    (cnts, _) = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse =True)[:10]
    print len(cnts)
    if len(cnts)==1:
        cnt1=cnts[0]
        x,y,w,h = cv2.boundingRect(cnt1)
        crop=image1[0:y+h+8,x:x+w]
        #crop the text
        #cv2.imshow('ghi',crop)
        #cv2.waitKey(0)
        img = cv2.medianBlur(crop,3)
        img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2) 
        #smooth the image
        #cv2.imshow('ghi',img)
        #cv2.waitKey(0)
        #img=inverte(img)
        #change black to white and vcie versa
        cv2.imshow('ghi',img)
        cv2.waitKey(0)
        img=Reshaping(img)
        #change the shape to 28*28
        cv2.imshow('ghi',img)
        cv2.waitKey(0)
        img=np.reshape(img, (784, 1))
        #make 1-d array
        result=net.predict(img)
        print result

    elif len(cnts)>1:
        cnts = sorted(cnts, key = cv2.contourArea, reverse =False)[:10]
        cnt1=cnts[0]
        x,y,w,h = cv2.boundingRect(cnt1)
        crop=image[0:y+h+8,x:x+w]
        img = cv2.medianBlur(crop,3)
        img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2) 
        #img=inverte(img)
        img=Reshaping(img)
        img=np.reshape(img, (784, 1))
        result=net.predict(img)
        print result

        cnt2=cnts[1]
        x,y,w,h = cv2.boundingRect(cnt2)
        crop=image[0:y+h+8,x:x+w]
        img = cv2.medianBlur(crop,3)
        img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2) 
        #img=inverte(img)
        img=Reshaping(img)
        img=np.reshape(img, (784, 1))
        result=net.predict(img)
        print result
    

more black content on image
1
3


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:122: RuntimeWarning: overflow encountered in exp


# Program ends here